### Create models to predict whether a client will default on their next payment. Your group should work together and produce an ensemble model and then analyze the effectiveness of your model.

### Write a short report explaining your best ensemble model and the evaluation of that model. Consider what metrics you should use to evaluate your model.  At a minimum you should report F1 scores, ROC curve, and AUC, but you might also want to consider using  accuracy scores, confusion matrices, recall, precision, sensitivity, specificity, false positives, false negatives, Matthews correlation coefficient etc.   Please justify the evaluation metric(s) that you use. 

### Submit your report as a pdf.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
clients = pd.read_csv('credit_clients.csv')

In [3]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ID                          30000 non-null  int64
 1   LIMIT_BAL                   30000 non-null  int64
 2   SEX                         30000 non-null  int64
 3   EDUCATION                   30000 non-null  int64
 4   MARRIAGE                    30000 non-null  int64
 5   AGE                         30000 non-null  int64
 6   PAY_0                       30000 non-null  int64
 7   PAY_2                       30000 non-null  int64
 8   PAY_3                       30000 non-null  int64
 9   PAY_4                       30000 non-null  int64
 10  PAY_5                       30000 non-null  int64
 11  PAY_6                       30000 non-null  int64
 12  BILL_AMT1                   30000 non-null  int64
 13  BILL_AMT2                   30000 non-null  int64
 14  BILL_A

ID: ID of each client 
LIMIT_BAL: Amount of given credit in NT dollars (includes individual and family/supplementary credit 
SEX: Gender (1=male, 2=female) 
EDUCATION: (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown) 
MARRIAGE: Marital status (1=married, 2=single, 3=others) 
AGE: Age in years 
PAY_0: Repayment status in September, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, … 8=payment delay for eight months, 9=payment delay for nine months and above) 
PAY_2: Repayment status in August, 2005 (scale same as above) 
PAY_3: Repayment status in July, 2005 (scale same as above) 
PAY_4: Repayment status in June, 2005 (scale same as above) 
PAY_5: Repayment status in May, 2005 (scale same as above) 
PAY_6: Repayment status in April, 2005 (scale same as above) 
BILL_AMT1: Amount of bill statement in September, 2005 (NT dollar) 
BILL_AMT2: Amount of bill statement in August, 2005 (NT dollar) 
BILL_AMT3: Amount of bill statement in July, 2005 (NT dollar) 
BILL_AMT4: Amount of bill statement in June, 2005 (NT dollar) 
BILL_AMT5: Amount of bill statement in May, 2005 (NT dollar) 
BILL_AMT6: Amount of bill statement in April, 2005 (NT dollar) 
PAY_AMT1: Amount of previous payment in September, 2005 (NT dollar) 
PAY_AMT2: Amount of previous payment in August, 2005 (NT dollar) 
PAY_AMT3: Amount of previous payment in July, 2005 (NT dollar) 
PAY_AMT4: Amount of previous payment in June, 2005 (NT dollar) 
PAY_AMT5: Amount of previous payment in May, 2005 (NT dollar) 
PAY_AMT6: Amount of previous payment in April, 2005 (NT dollar) 
default.payment.next.month: Default payment (1=yes, 0=no)

In [4]:
clients.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
#df = df.fillna(df.mean())
x = clients.iloc[:,1:23]
y = clients.iloc[:,24]
poly = PolynomialFeatures(interaction_only=True)
x = poly.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [6]:
lnr_model = LogisticRegression(solver='lbfgs')

In [7]:
lnr_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
lnr_model.predict(x_test)
print(lnr_model.score(x_test,y_test))
print(lnr_model.score(x_train, y_train))

0.7645
0.7748333333333334


In [9]:
clf = Lasso(alpha=0.01)
clf.fit(x_train, y_train)
clf.predict(x_test)
print(clf.score(x_test,y_test))
print(clf.score(x_train,y_train))

0.158737756542506
0.15414711734412123


In [10]:
from sklearn.ensemble import RandomForestClassifier
#create a new random forest classifier
forest = RandomForestClassifier(n_estimators = 100,verbose=3,n_jobs=-1)

In [11]:
forest.fit(x_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100building tree 3 of 100

building tree 4 of 100building tree 5 of 100
building tree 6 of 100
building tree 7 of 100

building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.4s


building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   14.1s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=3,
                       warm_start=False)

In [12]:
print(forest.score(x_test, y_test))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


0.82


In [13]:
print(forest.score(x_train, y_train))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.0s


0.9993333333333333


[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


In [ ]:
#conda install -c conda-forge xgboost

In [ ]:
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
test_predictions = [round(value) for value in y_pred]
y_pred = xgb.predict(x_train)
train_predictions = [round(value) for value in y_pred]
print("Training Accuracy:", accuracy_score(train_predictions, y_train)) #Accuracy of the model when training.
print("Testing Accuracy:", accuracy_score(test_predictions, y_test)) # Accuracy of the test.
final_performances.append(accuracy_score(test_predictions,y_test))
final_algs.append("XGBoost")

In [ ]:
knn_pipe = Pipeline(steps = [("feature",PolynomialFeatures(interaction_only=True)),
                         ("scalar",StandardScaler()),("model",KNeighborsRegressor(n_jobs= -1))])
knn_pipe.fit(x_train,y_train)

In [ ]:
knn_pipe.score(x_test, y_test)

In [ ]:
from sklearn.ensemble import VotingClassifier
vc = VotingClassifier(estimators=[('xgb', xgb), ('lr', lnr_model), ('knn', knn_pipe), ('forest', forest)], voting='hard', n_jobs = -1)
vc.fit(x_train, y_train)
print("Training Accuracy:",vc.score(x_train, y_train)) # Accuracy of the model when training.
print("Testing Accuracy:", vc.score(x_test, y_test) ) # Accuracy of the test.
final_performances.append(vc.score(x_test,y_test))
final_algs.append("Voting")

### F1